In [12]:
def draw_line(mask, color=(1,0,0), width=3):
    mapimg = (mask == 0).astype(int)

    ver_seg = np.where(mapimg[:,1:] != mapimg[:,:-1])
    hor_seg = np.where(mapimg[1:,:] != mapimg[:-1,:])


    l = []
    for p in zip(*hor_seg):
        l.append((p[1], p[0]+1))
        l.append((p[1]+1, p[0]+1))
        l.append((np.nan,np.nan))

    # and the same for vertical segments
    for p in zip(*ver_seg):
        l.append((p[1]+1, p[0]))
        l.append((p[1]+1, p[0]+1))
        l.append((np.nan, np.nan))


    segments = np.array(l)

    x0=0
    x1=224
    y0=0
    y1=224

    segments[:,0] = x0 + (x1-x0) * segments[:,0] / mapimg.shape[1]
    segments[:,1] = y0 + (y1-y0) * segments[:,1] / mapimg.shape[0]
    
    plt.plot(segments[:,0], segments[:,1], color=color, linewidth=width)

In [13]:
import torch
from torch.autograd import Variable
from torchvision import models
import cv2
import sys
import numpy as np

from matplotlib import pyplot as plt
#%matplotlib inline

use_cuda = torch.cuda.is_available()
FloatTensor = torch.cuda.FloatTensor if use_cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if use_cuda else torch.LongTensor
Tensor = FloatTensor

def tv_norm(input, tv_beta):
    img = input[0, 0, :]
    row_grad = torch.mean(torch.abs((img[:-1 , :] - img[1 :, :])).pow(tv_beta))
    col_grad = torch.mean(torch.abs((img[: , :-1] - img[: , 1 :])).pow(tv_beta))
    return row_grad + col_grad

def preprocess_image(img):
    means=[0.485, 0.456, 0.406]
    stds=[0.229, 0.224, 0.225]

    preprocessed_img = img.copy()[: , :, ::-1]
    for i in range(3):
        preprocessed_img[:, :, i] = preprocessed_img[:, :, i] - means[i]
        preprocessed_img[:, :, i] = preprocessed_img[:, :, i] / stds[i]
    preprocessed_img = \
        np.ascontiguousarray(np.transpose(preprocessed_img, (2, 0, 1)))

    if use_cuda:
        preprocessed_img_tensor = torch.from_numpy(preprocessed_img).cuda()
    else:
        preprocessed_img_tensor = torch.from_numpy(preprocessed_img)

    preprocessed_img_tensor.unsqueeze_(0)
    return Variable(preprocessed_img_tensor, requires_grad = False)

def sv_res(path, content, area_mask = None, do_mask = False):
    plt.ioff()
    fig = plt.figure()
    plt.imshow(np.uint8(255*content))
    if do_mask: 
        draw_line(area_mask, width=4, color=(0,0,0.7))
    fig.savefig(path, bbox_inches='tight')
    
def save(name, mask, img, blurred, area_mask = None, do_mask = False, full_mask = False):
    mask = mask.cpu().data.numpy()[0]
    mask = np.transpose(mask, (1, 2, 0))

   # mask = (mask - np.min(mask)) / np.max(mask)
    mask = 1 - mask
    heatmap = cv2.applyColorMap(np.uint8(255*mask), cv2.COLORMAP_JET)
    
    heatmap = np.float32(heatmap) / 255
    cam = 1.0*heatmap + np.float32(img)/255
    cam = cam / np.max(cam)

    img = np.float32(img) / 255
    perturbated = np.multiply(1 - mask, img) + np.multiply(mask, blurred)    

    pre_name = './results-clf/' + name + '/' + name
    if do_mask:
        pre_name = pre_name + '-mask'
    if full_mask:
        pre_name = pre_name + '-full'
        
    cv2.imwrite(pre_name +  "-perturbated.png", np.uint8(255*perturbated))
    sv_res(pre_name + "-heatmap.png", heatmap, area_mask, do_mask)
    cv2.imwrite(pre_name + "-mask.png", np.uint8(255*mask))
    sv_res(pre_name + "-cam.png", cam, area_mask, do_mask)

def numpy_to_torch(img, requires_grad = True):
    if len(img.shape) < 3:
        output = np.float32([img])
    else:
        output = np.transpose(img, (2, 0, 1))

    output = torch.from_numpy(output)
    if use_cuda:
        output = output.cuda()

    output.unsqueeze_(0)
    v = Variable(output, requires_grad = requires_grad)
    return v

def load_model():
    model = models.vgg19(pretrained=True)
    #model = models.segmentation.deeplabv3_resnet101(pretrained=True, progress=True, num_classes=21)    
    model.eval()
    if use_cuda:
        model.cuda()
    
    for p in model.features.parameters():
        p.requires_grad = False
    for p in model.classifier.parameters():
        p.requires_grad = False

    return model

In [14]:
def get_area_mask(do_mask = False, mask = "dog", full_mask = False):
    if (not do_mask): return np.zeros((224, 224), dtype = np.float32)
    if mask == "dog":
        if not full_mask:
            return np.load('./faces/dog-mask.npy')
        return np.load('./full/dog-full.npy')
    
    if mask == "cat":
        if not full_mask:
            return np.load('./faces/cat-mask.npy')
        return np.load('./full/cat-full.npy')
    
    if mask == "person":
        if not full_mask:
            return np.load('./faces/person-mask.npy')
        return np.load('./full/person-full.npy')

In [15]:
def explain_classification(name, do_mask = False, full_mask = False):
    tv_beta = 3
    learning_rate = 0.1
    max_iterations = 500
    l1_coeff = 0.01
    tv_coeff = 0.2
    image_path = "./original-images/" + name + ".png"

    model = load_model()
    original_img = cv2.imread(image_path, 1)
    original_img = cv2.resize(original_img, (224, 224))
    img = np.float32(original_img) / 255

    blurred_img_init = np.float32(cv2.medianBlur(original_img, 11))/255

    # Convert to torch variables
    img = preprocess_image(img)
    blurred_img = preprocess_image(blurred_img_init)

    mask_init = np.ones((28, 28), dtype = np.float32)
    mask = numpy_to_torch(mask_init)

    area_mask = get_area_mask(do_mask, name, full_mask)
    mask_static = numpy_to_torch(area_mask, requires_grad = False)

    upsample = torch.nn.UpsamplingBilinear2d(size=(224, 224)).cuda()

    optimizer = torch.optim.Adam([mask], lr=learning_rate)

    target = torch.nn.Softmax()(model(img))
    category = np.argmax(target.cpu().data.numpy())
    print("-------------------")
    print("IMAGE:" + name)
    print("-------------------")
    print("Category with highest probability")
    print(category)
    print("Optimizing.. ")
    print("Iterations:")

    for i in range(max_iterations):
        if i%100 == 0: 
            print(i)

        def_mask = torch.mul((1-mask_static), upsample(mask))

        upsampled_mask = def_mask
        upsampled_mask = upsampled_mask.expand(1, 3, upsampled_mask.size(2),  upsampled_mask.size(3))
        
        # Use the mask to perturbated the input image.
        perturbated_input = img.mul(upsampled_mask) + blurred_img.mul(1-upsampled_mask)
        
           
        outputs = torch.nn.Softmax()(model(perturbated_input))
        
        loss = (
            l1_coeff*torch.mean(torch.abs(1 - def_mask)) + 
            ((outputs[0, category] - target[0, category])/ 
             torch.mean(torch.abs(1 - mask_static)))
        )

        optimizer.zero_grad()
        loss.backward()

        optimizer.step()

        mask.data.clamp_(0, 1)

    upsampled_mask = upsample(def_mask)
    save(name, upsampled_mask, original_img, blurred_img_init, area_mask, do_mask, full_mask)
     
    print("--------------END--------------")

In [16]:
import os
img_names = ["dog", "person"]
for name in img_names:
    try:
        os.mkdir("results-clf/" + name)
    except:
        pass

    explain_classification(name, do_mask = True, full_mask = True)
    explain_classification(name, do_mask = True)    
    explain_classification(name, do_mask = False)

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:30: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:53: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


-------------------
IMAGE:dog
-------------------
Category with highest probability
245
Optimizing.. 
Iterations:
0
100
200
300
400
--------------END--------------
-------------------
IMAGE:dog
-------------------
Category with highest probability
245
Optimizing.. 
Iterations:
0
100
200
300
400
--------------END--------------
-------------------
IMAGE:dog
-------------------
Category with highest probability
245
Optimizing.. 
Iterations:
0
100
200
300
400
--------------END--------------
-------------------
IMAGE:person
-------------------
Category with highest probability
902
Optimizing.. 
Iterations:
0
100
200
300
400
--------------END--------------
-------------------
IMAGE:person
-------------------
Category with highest probability
902
Optimizing.. 
Iterations:
0
100
200
300
400


/usr/local/lib/python3.5/dist-packages/matplotlib/pyplot.py:537: RuntimeWarning: More than 20 figures have been opened. Figures created through the pyplot interface (`matplotlib.pyplot.figure`) are retained until explicitly closed and may consume too much memory. (To control this warning, see the rcParam `figure.max_open_warning`).
  max_open_warning, RuntimeWarning)


--------------END--------------
-------------------
IMAGE:person
-------------------
Category with highest probability
902
Optimizing.. 
Iterations:
0
100
200
300
400
--------------END--------------
